In [1]:
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_runner
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import os
import itertools
import glob
import dataclasses

MainProcess


In [ ]:
# Prepare datasets
fls.download_kaggle_dataset('jeroencottaar/byu-many-models-abbreviated/', fls.result_dir + '/many_abbr_res/')

In [3]:
files = glob.glob(fls.result_dir + '/many_abbr_res/' + '*.pickle')
for f in files:
    print(f)
    data = fls.dill_load('d:/flagellar/results//many_abbr_res\Baseline_0_0d8fae03_a.pickle')

d:/flagellar/results//many_abbr_res\Baseline_0_0d8fae03_a.pickle


AttributeError: Can't get attribute '_get_slots' on <module 'dataclasses' from 'C:\\Users\\jeroe\\anaconda3\\envs\\jupyterlab-debugger2\\lib\\dataclasses.py'>